# 🤖 Notebook: model_training.ipynb

**Goal:** Train and evaluate a Linear Regression baseline.

### Include
- Load `data/cleaned_pulsebat.csv`
- Train/test split
- Fit `LinearRegression`
- Metrics: R², MSE, MAE
- Plot: Predicted vs Actual SOH
- Save model to `models/soh_linear_model.pkl` (joblib)

### Deliverable
- Saved model + performance plots


In [2]:
# Task 1 – Train Linear Regression Model
# --------------------------------------
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

# --- Resolve paths whether running from project root or /notebooks ---
if os.path.basename(os.getcwd()) == "notebooks":
    DATA_PATH = "../data/cleaned_pulsebat.csv"
    MODEL_DIR = "../models"
else:
    DATA_PATH = "data/cleaned_pulsebat.csv"
    MODEL_DIR = "models"

print("Loading data from:", DATA_PATH)

# 1) Load cleaned dataset
df = pd.read_csv(DATA_PATH)

# 2) Separate features (U1–U21) and target (SOH)
feature_cols = [f"U{i}" for i in range(1, 22)]
target_col = "SOH"

X = df[feature_cols].values
y = df[target_col].values

# 3) Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Shapes → X_train: {X_train.shape}, X_test: {X_test.shape}, "
      f"y_train: {y_train.shape}, y_test: {y_test.shape}")

# 4) Train Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

print("✅ Model trained.")
print("Intercept:", model.intercept_)
print("Coefficients (len):", len(model.coef_))

# 5) (Optional here) Save the trained model for later steps
os.makedirs(MODEL_DIR, exist_ok=True)
model_path = os.path.join(MODEL_DIR, "soh_linear_model.pkl")
joblib.dump(model, model_path)
print(f"💾 Saved model → {model_path}")


Loading data from: ../data/cleaned_pulsebat.csv
Shapes → X_train: (536, 21), X_test: (134, 21), y_train: (536,), y_test: (134,)
✅ Model trained.
Intercept: 4.86392140717598
Coefficients (len): 21
💾 Saved model → ../models/soh_linear_model.pkl
